In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import urllib.request
from urllib.request import urlopen
import time
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import urllib
import socket, whois

In [49]:
def long_strings_count(url) :
    options = Options()
    options.add_argument("--headless")
    options.add_argument('/home/archit/sem2/ai/chromedriver/chromedriver')

    browser = webdriver.Chrome('/home/archit/sem2/ai/chromedriver/chromedriver', options=options)
    browser.get("http://" + url)
    soup = BeautifulSoup(browser.page_source, "html.parser")
    scripts = soup.findAll('script')
    
    count = 0
    total_avg = 0
    total_count = 0 
    max_len_string = 0
    above_avg_count =  0 
    
    for sc in scripts:
        check = False
        try :
            src = sc['src']
        except :
            src = None 
        
        if src is None :
            txt = sc.text
        else :
            try :
                txt = urlopen(src).read().decode('utf-8')
            except :
                txt = None
                check = True
        
        if txt is None : 
            continue 

        result1 = re.findall(r'"(.*?)"', txt)
        result2 = re.findall(r"'(.*?)'", txt)
        
        result = result1 + result2
        
        try :
            temp_avg = sum( map(len, result)) 
            temp_count =  len(result)
            max_len_string = max( max_len_string , len(max(result, key=len)))
        except :
            temp_avg = 0
            temp_count =  0
            max_len_string = max_len_string
        
        total_avg += temp_avg
        total_count  += temp_count
        
  
        
        try : 
            for ele in result :
                if len(ele) > temp_avg/temp_count :
                    above_avg_count += 1
        except :
            above_avg_count = above_avg_count
        
    return total_count , total_avg , max_len_string , above_avg_count
        

    
# #         for ele in result1 :
# #             if len(ele) >= 100 :
# #                 count += 1
# #         for ele in result2 :
# #             if len(ele) >= 100 :
# #                 count += 1
    
#     return count

In [3]:
def find_list_resources(tag, attribute,soup):
    list = []
    for x in soup.findAll(tag):
        try:
            list.append(x[attribute])
        except KeyError:
            pass
    return(list)


def iframe_count(url):
    try :
        response = requests.get(url)
    except : 
        response = None
        
    if response is None :
        return None
    
    soup = BeautifulSoup(response.text, "html.parser")
    return len(soup.findAll('iframe'))
    
    
        
def scripts_count(url) :
    try :
        response = requests.get(url)
    except : 
        response = None
        
    if response is None :
        return None
    
    soup = BeautifulSoup(response.text, "html.parser")
    return len(soup.findAll('script'))


def external_url_count(url) :
    try :
        response = requests.get(url)
    except : 
        response = None
    
    if response is None :
        return None
    
    soup = BeautifulSoup(response.text, "html.parser")
    count = 0
    count += len(find_list_resources('img',"src",soup))   
    count += len(find_list_resources('script',"src",soup))    
    count += len(find_list_resources("link","href",soup))
    count += len(find_list_resources("video","src",soup))         
    count += len(find_list_resources("audio","src",soup)) 
    count += len(find_list_resources("iframe","src",soup))
    count += len(find_list_resources("embed","src",soup))
    count += len(find_list_resources("object","data",soup))         
    count += len(find_list_resources("source","src",soup))
    return count

In [4]:
def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)'  
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    if match:
        # print match.group()
        return 1
    else:
        # print 'No matching pattern found'
        return 0

In [5]:
def shortening_service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return 1
    else:
        return 0

In [6]:
def https_token(url):
    match = re.search('https://|http://', url)
    if match==None:
        return 0
    if match.start(0) == 0:
        url = url[match.end(0):]
    match = re.search('http|https', url)
    if match:
        return 0
    else:
        return 1

In [7]:
import socket
import requests as req
import re
import time

def dns_response(url):
    try :
        url = url.split("/")
        hostname = url[0]
    except : 
        print(" issue in finding host name from url : " + url)
        return -2
    
    try :
        startTime = time.time()
        ip = socket.gethostbyname(hostname)
        endTime = time.time()
        diff = (endTime - startTime)* 1000
        return diff
    except :
        return -1

In [8]:
from urllib.parse import urlparse
import time
import ssl
import urllib.request  as urllib2
import urllib

def networks(link):
    if not urlparse(link).scheme:
        url = "http://" + link
    gcontext = ssl._create_unverified_context()
    handshake_start = time.time()
    http_status_code = 400

    try : 
        stream = urllib2.urlopen(url,context=gcontext)
        http_status_code = stream.getcode()

    except urllib.error.HTTPError  as e :
            http_status_code = e.code
    
    
    if http_status_code >= 400 :
        return 0 , 0 , 0 , False , 'http' , http_status_code , None      
            
        
    new_url = stream.geturl()
    
    try : 
        server_name = stream.info()["Server"]
    except :
        server_name = None
    
    redirect = False
    protocol =  "http"
    
    if new_url :
        new_url = new_url.split("://")
        protocol =  new_url[0]
        del new_url[0]
        new_url = "://".join(new_url)
        
        if new_url != link:
            redirect = True

            
        
#     print(stream.info())
    handshake_end = time.time()
    handshake_time = (handshake_end - handshake_start) * 1000
#     print ('HTTP handshake time = %.5f ms' %handshake_time)
    data_start = time.time()
    data_length = len(stream.read())
    data_end = time.time()
    data_time = (data_end - data_start)*1000
#     print("data time = %.5f ms" %data_time)
#     print(data_length)
    return handshake_time, data_time, data_length , redirect , protocol , http_status_code,server_name
# networks("twitter.com/aQuariusrecOrds")

In [9]:
#average_len, count and length_of_largest token

def avg_large_count(string):
    avg = 0.0
    largest = 0
    string = string.strip()
    tokens = re.split('\W+|_', string)
    if string == "":
        return [0,0,0]
    else:
        if "" in tokens:
            count = len(tokens)-1
        else:
            count = len(tokens)
        for token1 in tokens:
            avg += len(token1)
            if largest < len(token1):
                largest = len(token1)
        avg /= count
    return count, avg, largest

In [10]:
#check .exe is present in url or not
def is_exe(url):
    if ".exe" in url:
        return 1 
    else:
        return 0

In [11]:
# count security sensitive words that are present in url

def Security_sensitive(url):

    sec_sen_words=['confirm', 'account', 'banking', 'secure', 'ebayisapi', 'webscr', 'login', 'signin']
    cnt=0
    for ele in sec_sen_words:
        if(ele in url):
            cnt+=1;

    return cnt

In [12]:
# get asn from host
def getASN(host):
    try:
        g = pygeoip.GeoIP('GeoIPASNum.dat')
        asn=int(g.org_by_name(host).split()[0][2:])
        return asn
    except:
        return  nf

In [13]:
# asn no. 
def whois(server, query):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect((server, 43))
    sock.send(query + '\r\n')
    response = ''
    while True:
        d = sock.recv(4096)
        response += d
        if d == '':
            break
    sock.close()
    return response
print(whois('whois.ra.net', 'AS15169'))

ConnectionRefusedError: [Errno 111] Connection refused

In [14]:
def no_of_dots(url) :
    return url.count(".")

In [15]:
# count host name length

def hostnamelen_and_digitcountdomain(url):
    spltAr = url.split("://");
    i = (0,1)[len(spltAr)>1];
    hl = spltAr[i].split("?")[0].split('/')[0].split(':')[0].lower();
    return len(hl) , len(hl) - hl.index('.') - 1

In [16]:
hostnamelen_and_digitcountdomain("www.facebook.com")

(16, 12)

## final data 

In [29]:
data = pd.read_csv('data.csv')
print(data.keys())
print(data.shape)

Index(['url', 'label'], dtype='object')
(420464, 2)


In [30]:
print(data.shape)
print(data.head())

(420464, 2)
                      url label
0  diaryofagameaddict.com   bad
1        espdesign.com.au   bad
2      iamagameaddict.com   bad
3           kalantzis.net   bad
4   slightlyoffcenter.net   bad


In [31]:
data["label"].value_counts()

good    344821
bad      75643
Name: label, dtype: int64

In [32]:
data = data.sample(frac=0.005, random_state=1)
data["label"].value_counts()

good    1730
bad      372
Name: label, dtype: int64

In [33]:
data.to_csv('final_data.csv')

## removing  the data with no dns info 

In [18]:
data = pd.read_csv('final_data.csv')
print(data.head())
data['label'].value_counts()

   Unnamed: 0                                                url label
0      155439                        twitter.com/aQuariusrecOrds  good
1      152415                                       tong464.org/  good
2      276521  legacy.com/obituaries/orlandosentinel/obituary...  good
3      305692                           natcath.org/ncr_onli.htm  good
4      147638  store02.prostores.com/servlet/omegaordnancewer...  good


good    1730
bad      372
Name: label, dtype: int64

### Adding url length field :  

In [19]:
data['url_length'] = data['url'].apply(lambda x: len(x))

In [20]:
data.head()

,Unnamed: 0,url,label,url_length
0,155439,twitter.com/aQuariusrecOrds,good,27
1,152415,tong464.org/,good,12
2,276521,legacy.com/obituaries/orlandosentinel/obituary...,good,92
3,305692,natcath.org/ncr_onli.htm,good,24
4,147638,store02.prostores.com/servlet/omegaordnancewer...,good,104


### Adding special Character count 

In [21]:
lst = []
for i in range(len(data)):
    new = re.sub('[\w]+' ,'', data.iloc[i]['url'])
    lst.append(len(new))
#     print(i)
data['special_character_count'] = lst

In [40]:
data.head()

,Unnamed: 0,url,label,url_length,special_character_count
0,155439,twitter.com/aQuariusrecOrds,good,27,2
1,152415,tong464.org/,good,12,2
2,276521,legacy.com/obituaries/orlandosentinel/obituary...,good,92,12
3,305692,natcath.org/ncr_onli.htm,good,24,3
4,147638,store02.prostores.com/servlet/omegaordnancewer...,good,104,13


### having ip address 

In [22]:
lst = []
for i in range(len(data)):
    lst.append(having_ip_address(data.iloc[i]["url"]))
data['having_ip_address'] = lst

In [23]:
data.head()

,Unnamed: 0,url,label,url_length,special_character_count,having_ip_address
0,155439,twitter.com/aQuariusrecOrds,good,27,2,0
1,152415,tong464.org/,good,12,2,0
2,276521,legacy.com/obituaries/orlandosentinel/obituary...,good,92,12,0
3,305692,natcath.org/ncr_onli.htm,good,24,3,0
4,147638,store02.prostores.com/servlet/omegaordnancewer...,good,104,13,0


In [24]:
data.to_csv("final_data1.csv")

### DNS response time (removing data with no dns time) 

In [25]:
lst  = []
for i in range(len(data)) : 
    print(i ,end= '\r')
    lst.append(dns_response(data.iloc[i]["url"]))
data['dns_response_time'] = lst

In [26]:
#removing url with no data url 
data = data[data.dns_response_time != -1]
data.head()


,Unnamed: 0,url,label,url_length,special_character_count,having_ip_address,dns_response_time
0,155439,twitter.com/aQuariusrecOrds,good,27,2,0,1.866579
1,152415,tong464.org/,good,12,2,0,654.555559
2,276521,legacy.com/obituaries/orlandosentinel/obituary...,good,92,12,0,57.567120
3,305692,natcath.org/ncr_onli.htm,good,24,3,0,1.390219
5,165286,wn.com/San_Diego_State_Aztecs,good,29,2,0,1.077890


In [27]:
print(data["label"].value_counts())

good    1598
bad      221
Name: label, dtype: int64


In [28]:
data.to_csv("final_data2.csv")

In [16]:
data = pd.read_csv("final_data2.csv")
data.head()

,Unnamed: 0,Unnamed: 0.1,url,label,url_length,special_character_count,having_ip_address,dns_response_time
0,0,155439,twitter.com/aQuariusrecOrds,good,27,2,0,1.866579
1,1,152415,tong464.org/,good,12,2,0,654.555559
2,2,276521,legacy.com/obituaries/orlandosentinel/obituary...,good,92,12,0,57.567120
3,3,305692,natcath.org/ncr_onli.htm,good,24,3,0,1.390219
4,5,165286,wn.com/San_Diego_State_Aztecs,good,29,2,0,1.077890


In [29]:
lst1 = []
lst2 = []
lst3 = []
lst4 = []
lst5 = []
lst6 = []
lst7 = []
exceptions = {}
for i in range(len(data)): 
    print(i ,end= '\r')
    count = 0 
    
    while count < 3:
        try :            
            handshake_time, data_time, data_length , redirect , protocol , http_status_code,server_name = networks(data.iloc[i]["url"])
            count  = 3
        except :
            if count is 2 :
                exceptions[i] = data.iloc[i]["url"]
                
            handshake_time, data_time, data_length , redirect , protocol , http_status_code,server_name = None, None, None, None, None, None, None
            print(str(i) + " : "+  data.iloc[i]["url"] + "\n")
            count  += 1 
            
        
        
    lst1.append(handshake_time)
    lst2.append(data_time)
    lst3.append(data_length)
    lst4.append(redirect)
    lst5.append(protocol)
    lst6.append(http_status_code)
    lst7.append(server_name)
    


32 : butlercounty.ky.gov/Cities/

64 : stuffaboutnames.com/peter/actors.htm

64 : stuffaboutnames.com/peter/actors.htm

64 : stuffaboutnames.com/peter/actors.htm

191 : xbox.ign.com/objects/566/566353.html

191 : xbox.ign.com/objects/566/566353.html

191 : xbox.ign.com/objects/566/566353.html

331 : idahostatesman.com/2011/11/20/1886338/college-of-idaho-advances-to-naia.html

331 : idahostatesman.com/2011/11/20/1886338/college-of-idaho-advances-to-naia.html

331 : idahostatesman.com/2011/11/20/1886338/college-of-idaho-advances-to-naia.html

383 : webhome.idirect.com/~letanu/baillon2/pafg67.htm

383 : webhome.idirect.com/~letanu/baillon2/pafg67.htm

383 : webhome.idirect.com/~letanu/baillon2/pafg67.htm

497 : wn.com/Marie-Élaine_Thibert

497 : wn.com/Marie-Élaine_Thibert

497 : wn.com/Marie-Élaine_Thibert

765 : videolala.com/actors/Ernest_Truex

765 : videolala.com/actors/Ernest_Truex

765 : videolala.com/actors/Ernest_Truex

785 : concertticketcenter.com/Event.aspx?EventID=43399

785 

In [30]:
data['handshake_time'] = lst1
data['data_time'] = lst2
data['data_length'] = lst3
data['redirect'] = lst4
data['protocol'] = lst5
data['http_status_code'] = lst6
data['server_name'] = lst7

In [31]:
data.to_csv("final_data3.csv")

In [36]:
data.head()

,Unnamed: 0,Unnamed: 0.1,url,label,url_length,special_character_count,having_ip_address,dns_response_time,handshake_time,data_time,data_length,redirect,protocol,http_status_code,server_name
0,0,155439,twitter.com/aQuariusrecOrds,good,27,2,0,1.866579,809.915781,185.235262,301912.0,False,https,200.0,tsa_k
1,1,152415,tong464.org/,good,12,2,0,654.555559,3379.443407,799.807549,162844.0,True,http,200.0,Apache
2,2,276521,legacy.com/obituaries/orlandosentinel/obituary...,good,92,12,0,57.567120,2735.617161,29.102325,129957.0,True,https,200.0,cloudflare
3,3,305692,natcath.org/ncr_onli.htm,good,24,3,0,1.390219,1009.941101,0.138521,3453.0,False,http,200.0,nginx/1.14.1
4,5,165286,wn.com/San_Diego_State_Aztecs,good,29,2,0,1.077890,6096.054077,350.750208,204520.0,False,https,200.0,nginx


In [37]:
exceptions

{64: 'stuffaboutnames.com/peter/actors.htm',
 191: 'xbox.ign.com/objects/566/566353.html',
 331: 'idahostatesman.com/2011/11/20/1886338/college-of-idaho-advances-to-naia.html',
 383: 'webhome.idirect.com/~letanu/baillon2/pafg67.htm',
 497: 'wn.com/Marie-Élaine_Thibert',
 765: 'videolala.com/actors/Ernest_Truex',
 785: 'concertticketcenter.com/Event.aspx?EventID=43399',
 844: 'blueridgecabinrentals.com/vacation-rental-home.asp?PageDataID=37621',
 963: 'kansascity.com/sports/mu/',
 972: 'stockhouse.com/News/USReleasesDetail.aspx?n=8350750',
 1016: 'uulyrics.com/music/france-joli/song-gonna-get-over-you/',
 1071: 'fbcity.com/hot-video-cat/theofania-papathoma-playboy-18811.html',
 1090: 'sunxiancheng.com/images/?http://us.battle.net/login/en/?ref=http://ljuytakus.battle.net/d3/en/index',
 1207: 'pivotalpayments.com/hd3/',
 1291: 'hockeyforum.com/international-hockey/19284-switzerland-2007-08-a.html',
 1312: 'cliqueclack.com/tv/2011/07/07/comic-con-james-warren-verne-langdon/',
 1335: 'prof

In [18]:
data = pd.read_csv("final_data3.csv")
data.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,url,label,url_length,special_character_count,having_ip_address,dns_response_time,handshake_time,data_time,data_length,redirect,protocol,http_status_code,server_name
0,0,0,155439,twitter.com/aQuariusrecOrds,good,27,2,0,1.866579,809.915781,185.235262,301912.0,False,https,200.0,tsa_k
1,1,1,152415,tong464.org/,good,12,2,0,654.555559,3379.443407,799.807549,162844.0,True,http,200.0,Apache
2,2,2,276521,legacy.com/obituaries/orlandosentinel/obituary...,good,92,12,0,57.567120,2735.617161,29.102325,129957.0,True,https,200.0,cloudflare
3,3,3,305692,natcath.org/ncr_onli.htm,good,24,3,0,1.390219,1009.941101,0.138521,3453.0,False,http,200.0,nginx/1.14.1
4,4,5,165286,wn.com/San_Diego_State_Aztecs,good,29,2,0,1.077890,6096.054077,350.750208,204520.0,False,https,200.0,nginx


In [19]:
lst  = []
for i in range(len(data)) : 
    print(i ,end= '\r')
    lst.append(shortening_service(data.iloc[i]["url"]))
data['shortening_service'] = lst

In [20]:
data.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,url,label,url_length,special_character_count,having_ip_address,dns_response_time,handshake_time,data_time,data_length,redirect,protocol,http_status_code,server_name,shortening_service
0,0,0,155439,twitter.com/aQuariusrecOrds,good,27,2,0,1.866579,809.915781,185.235262,301912.0,False,https,200.0,tsa_k,0
1,1,1,152415,tong464.org/,good,12,2,0,654.555559,3379.443407,799.807549,162844.0,True,http,200.0,Apache,0
2,2,2,276521,legacy.com/obituaries/orlandosentinel/obituary...,good,92,12,0,57.567120,2735.617161,29.102325,129957.0,True,https,200.0,cloudflare,0
3,3,3,305692,natcath.org/ncr_onli.htm,good,24,3,0,1.390219,1009.941101,0.138521,3453.0,False,http,200.0,nginx/1.14.1,0
4,4,5,165286,wn.com/San_Diego_State_Aztecs,good,29,2,0,1.077890,6096.054077,350.750208,204520.0,False,https,200.0,nginx,0


In [ ]:
lst1 = []
lst2 = []
lst3 = []
lst4 = []
for i in range(len(data)) :
    print(i , end = '\r')
    count = 0  
    while count < 3 :
        try :
            total_count , total_avg , max_len_string , above_avg_count = long_strings_count(data.iloc[i]["url"])
            count =  3
        except :
            print(str(i) + " : " + data.iloc[i]["url"])
            total_count , total__avg , max_len_string , above_avg_count = None, None, None, None
            count += 1
        print(total_count,  total_avg , max_len_string , above_avg_count)
    lst1.append(total_count)
    lst2.append(total_avg)
    lst3.append(max_len_string)
    lst4.append(above_avg_count)

0 0 0 0
1 : tong464.org/
None 0 None None
4208 None 5491 1061
2 : legacy.com/obituaries/orlandosentinel/obituary.aspx?n=william-c-underwood-bill&pid=147765497
None None None None


In [13]:
data = pd.read_csv('final_data3.csv')

In [14]:
data.head(

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,url,label,url_length,special_character_count,having_ip_address,dns_response_time,handshake_time,data_time,data_length,redirect,protocol,http_status_code,server_name
0,0,0,155439,twitter.com/aQuariusrecOrds,good,27,2,0,1.866579,809.915781,185.235262,301912.0,False,https,200.0,tsa_k
1,1,1,152415,tong464.org/,good,12,2,0,654.555559,3379.443407,799.807549,162844.0,True,http,200.0,Apache
2,2,2,276521,legacy.com/obituaries/orlandosentinel/obituary...,good,92,12,0,57.567120,2735.617161,29.102325,129957.0,True,https,200.0,cloudflare
3,3,3,305692,natcath.org/ncr_onli.htm,good,24,3,0,1.390219,1009.941101,0.138521,3453.0,False,http,200.0,nginx/1.14.1
4,4,5,165286,wn.com/San_Diego_State_Aztecs,good,29,2,0,1.077890,6096.054077,350.750208,204520.0,False,https,200.0,nginx


In [47]:
r_data = pd.read_csv('temp_data.csv')
len(r_data)

1819

In [44]:
long_strings_count("twitter.com/aQuariusrecOrds")

(26397, 63.02288138803652, 46438, 2436)

In [41]:
long_strings_count("tong464.org/	")

(9289, 11.865216923242546, 5491, 2936)

In [42]:
long_strings_count("legacy.com/obituaries/orlandosentinel/obituary...")

(220, 9.277272727272727, 45, 139)

In [50]:
long_strings_count("artists.letssingit.com/zaho-lkczm/albums")

(0, 0, 0, 0)

In [51]:
long_strings_count("fortunatogroupservice.com/includes/https:/aten...")

(0, 0, 0, 0)

In [42]:
lst1 = []
lst2 = []
lst3 = []
lst4 = []
for i in range(2) :
    print(i , end = '\r')
    count = 0  
    while count < 3 :
        try :
            total_count , total_avg , max_len_string , above_avg_count = long_strings_count(data.iloc[i]["url"])
            count =  3
        except :
            print(str(i) + " : " + data.iloc[i]["url"])
            total_count , total__avg , max_len_string , above_avg_count = None, None, None, None
            count += 1
    print(total_count,  total_avg , max_len_string , above_avg_count)
    lst1.append(total_count)
    lst2.append(total_avg)
    lst3.append(max_len_string)
    lst4.append(above_avg_count)

NameError: name 'data' is not defined

In [44]:
r_data["total_strings_count"] = lst1
r_data["average_string_length"] = lst2
r_data["max_len_strings"] = lst3
r_data["strings_above_avglength"] = lst4

In [45]:
r_data.to_csv("final_data5.csv",index = False)

In [57]:
data  = pd.read_csv("final_data5.csv")

In [58]:
data.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,url,label,url_length,special_character_count,having_ip_address,dns_response_time,...,protocol,http_status_code,server_name,scripts_count,iframe_count,external_link_count,total_strings_count,average_string_length,max_len_strings,strings_above_avglength
0,0,0,0,155439,twitter.com/aQuariusrecOrds,good,27,2,0,1.866579,...,https,200.0,tsa_k,8.0,6.0,73.0,26397.0,63.022881,46438.0,2436.0
1,1,1,1,152415,tong464.org/,good,12,2,0,654.555559,...,http,200.0,Apache,33.0,0.0,18.0,9289.0,11.865217,5491.0,2936.0
2,2,2,2,276521,legacy.com/obituaries/orlandosentinel/obituary...,good,92,12,0,57.567120,...,https,200.0,cloudflare,55.0,2.0,58.0,22093.0,44.528946,28934.0,4508.0
3,3,3,3,305692,natcath.org/ncr_onli.htm,good,24,3,0,1.390219,...,http,200.0,nginx/1.14.1,1.0,0.0,1.0,8.0,8.000000,8.0,0.0
4,4,4,5,165286,wn.com/San_Diego_State_Aztecs,good,29,2,0,1.077890,...,https,200.0,nginx,85.0,4.0,41.0,8248.0,12.376940,5491.0,2477.0


In [95]:
# count = 0
for i in range(len(data)):
    
    if str(data.iloc[i,23]) ==  'nan':        
        print(str(i) + " : " + data.iloc[i]["url"] , end =  '\r')
        try :
            total_count , total_avg , max_len_string , above_avg_count = long_strings_count(data.iloc[i]["url"])
        except :
            total_count , total_avg , max_len_string , above_avg_count = None, None, None, None
        if total_count :
            total_avg =  total_avg/total_count
            
#         print(total_count , total_avg , max_len_string , above_avg_count)
#         print(data.iloc[i,[20 ,21,22,23]])
        data.iloc[i,[20 ,21,22,23]] =  total_count , total_avg , max_len_string , above_avg_count
#         print(data.iloc[i,[20 ,21,22,23]])
        # print(count)

In [96]:
data.to_csv("final_data6.csv" , index =  False)